# EDA

EDA of the features and target

# Goal

Find patterns on the features for target

# Plan

Checklist what will be done on the notebook :

    [ ] Get Data
    [ ] Data Preparations
    [ ] Features Summary
    [ ] Features correlations to Target



In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import numpy as np

In [2]:
with open("../../config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

PATH_RAW = config['path']['raw']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']
PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'
PATH_UDF = config['path']['udf']

## Get Data

In [51]:
raw_trn = pd.read_csv(PATH_INTERIM+'data_trn_iter1.csv')

In [52]:
data_train = raw_trn.sample(int(len(raw_trn)*0.85), random_state = 124)
data_eval = raw_trn[~(raw_trn['ids'].isin(data_train['ids']))]

In [53]:
## check target
data_train['y'].value_counts(normalize = True)

no     0.884396
yes    0.115604
Name: y, dtype: float64

## Data Preparation
source : [link](https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy#ch5)

Creating
Cleaning
Combining
Completing
Creating
Converting

In [54]:
## separate the columns
col_num = ['age','balance','day',
                      'duration','pdays']
col_cat = ['job','contact','marital','default']

#### Missing Value

In [55]:
data_train

,ids,age,balance,day,duration,pdays,job,contact,marital,default,y
11493,14352,53,-24,14,319,-1,retired,cellular,divorced,no,no
25034,31327,49,1807,12,229,-1,technician,cellular,divorced,no,yes
11682,14580,46,311,15,232,-1,services,cellular,married,no,no
20668,25831,55,563,19,164,-1,management,cellular,married,no,no
7754,9711,39,575,6,109,-1,management,unknown,married,no,no
...,...,...,...,...,...,...,...,...,...,...,...
5853,7302,51,5801,29,147,-1,services,unknown,married,no,no
12581,15726,32,2196,21,1156,-1,services,cellular,married,no,no
23298,29153,55,621,2,114,208,management,cellular,married,no,no
31621,39483,50,641,25,128,-1,services,cellular,divorced,no,no


In [56]:
data_train.isnull().sum()

ids         0
age         0
balance     0
day         0
duration    0
pdays       0
job         0
contact     0
marital     0
default     0
y           0
dtype: int64

#### Outliers

In [57]:
## Numerical
df_num = data_train[col_num]

In [58]:
df_num.describe()

,age,balance,day,duration,pdays
count,30743.000000,30743.000000,30743.000000,30743.000000,30743.000000
mean,40.974758,1346.281528,15.820512,258.628338,40.335686
std,10.598649,3019.700987,8.315038,258.048992,100.194404
min,18.000000,-8019.000000,1.000000,0.000000,-1.000000
25%,33.000000,70.000000,8.000000,103.000000,-1.000000
50%,39.000000,444.000000,16.000000,181.000000,-1.000000
75%,48.000000,1412.000000,21.000000,318.000000,-1.000000
max,95.000000,102127.000000,31.000000,3881.000000,871.000000


In [59]:
## Cap the data

In [80]:
df_num.quantile(0.98).to_dict()

{'age': 63.0,
 'balance': 9330.599999999999,
 'day': 30.0,
 'duration': 1056.0,
 'pdays': 360.0}

In [60]:
rule_cap = df_num.quantile(0.98).to_dict()
joblib.dump(rule_cap, PATH_MODEL+'outliers_cap_rules.pkl')
rule_cap_load = joblib.load(PATH_MODEL+'outliers_cap_rules.pkl')

In [61]:
data_train['balance2'] = np.where(data_train['balance']<0,0,data_train['balance'])
col_num2 = []

for c2 in col_num:
    if c2 == 'balance':
        ck = c2+'2'
    else:
        ck = c2
    max_cap = rule_cap_load[c2]
    data_train[ck+'_cap'] = np.where(data_train[ck] >= max_cap, max_cap, data_train[ck])
    
    col_num2.append(ck+'_cap')

In [71]:
## check pdays
# df_num['pdays'].value_counts(normalize = True) --> 80% is -1

data_train['is_contacted'] = (data_train['pdays'] != -1).astype(int)


In [72]:
## Categoric
df_cat = data_train[col_cat]

In [74]:
df_cat.describe()

,job,contact,marital,default
count,30743,30743,30743,30743
unique,12,3,3,2
top,blue-collar,cellular,married,no
freq,6615,19897,18547,30192


In [79]:
for c in col_cat:
    print('***',c,'***')
    print(df_cat[c].value_counts(normalize = True))
    print('---')

*** job ***
blue-collar      0.215171
management       0.206519
technician       0.167550
admin.           0.115408
services         0.093290
retired          0.049507
self-employed    0.035683
entrepreneur     0.034317
housemaid        0.028299
unemployed       0.027811
student          0.020492
unknown          0.005953
Name: job, dtype: float64
---
*** contact ***
cellular     0.647204
unknown      0.287838
telephone    0.064958
Name: contact, dtype: float64
---
*** marital ***
married     0.603292
single      0.281462
divorced    0.115246
Name: marital, dtype: float64
---
*** default ***
no     0.982077
yes    0.017923
Name: default, dtype: float64
---


#### Features Engineering
(it can be done on EDA too)

#### Features Selection

## Conclusions